# Section: Federated Learning

# Lesson: Introducing Federated Learning

Federated Learning is a technique for training Deep Learning models on data to which you do not have access. Basically:

Federated Learning: Instead of bringing all the data to one machine and training a model, we bring the model to the data, train it locally, and merely upload "model updates" to a central server.

Use Cases:

    - app company (Texting prediction app)
    - predictive maintenance (automobiles / industrial engines)
    - wearable medical devices
    - ad blockers / autotomplete in browsers (Firefox/Brave)
    
Challenge Description: data is distributed amongst sources but we cannot aggregated it because of:

    - privacy concerns: legal, user discomfort, competitive dynamics
    - engineering: the bandwidth/storage requirements of aggregating the larger dataset

# Lesson: Introducing / Installing PySyft

In order to perform Federated Learning, we need to be able to use Deep Learning techniques on remote machines. This will require a new set of tools. Specifically, we will use an extensin of PyTorch called PySyft.

### Install PySyft

The easiest way to install the required libraries is with [Conda](https://docs.conda.io/projects/conda/en/latest/user-guide/overview.html). Create a new environment, then install the dependencies in that environment. In your terminal:

```bash
conda create -n pysyft python=3
conda activate pysyft # some older version of conda require "source activate pysyft" instead.
conda install jupyter notebook
pip install syft
pip install numpy
```

If you have any errors relating to zstd - run the following (if everything above installed fine then skip this step):

```
pip install --upgrade --force-reinstall zstd
```

and then retry installing syft (pip install syft).

If you are using Windows, I suggest installing [Anaconda and using the Anaconda Prompt](https://docs.anaconda.com/anaconda/user-guide/getting-started/) to work from the command line. 

With this environment activated and in the repo directory, launch Jupyter Notebook:

```bash
jupyter notebook
```

and re-open this notebook on the new Jupyter server.

If any part of this doesn't work for you (or any of the tests fail) - first check the [README](https://github.com/OpenMined/PySyft.git) for installation help and then open a Github Issue or ping the #beginner channel in our slack! [slack.openmined.org](http://slack.openmined.org/)

# Lesson: Basic Remote Execution in PySyft

## PySyft => Remote PyTorch

The essence of Federated Learning is the ability to train models in parallel on a wide number of machines. Thus, we need the ability to tell remote machines to execute the operations required for Deep Learning.

Thus, instead of using Torch tensors - we're now going to work with **pointers** to tensors. Let me show you what I mean. First, let's create a "pretend" machine owned by a "pretend" person - we'll call him Bob.

# Project: Playing with Remote Tensors

In this project, I want you to .send() and .get() a tensor to TWO workers by calling .send(bob,alice). This will first require the creation of another VirtualWorker called alice.

In [1]:
# try this project here!
import torch as torch
import syft as syft
hook = syft.TorchHook(torch)

W0626 17:29:39.326602 140523859060544 secure_random.py:26] Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was '/home/sijoonlee/anaconda3/envs/pysyft/lib/python3.7/site-packages/tf_encrypted/operations/secure_random/secure_random_module_tf_1.14.0.so'
W0626 17:29:39.382165 140523859060544 deprecation_wrapper.py:119] From /home/sijoonlee/anaconda3/envs/pysyft/lib/python3.7/site-packages/tf_encrypted/session.py:26: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.



In [2]:
bob = syft.VirtualWorker(hook, id="bob")

In [3]:
x = torch.tensor([1,2,3,4,5])
y = torch.tensor([1,1,1,1,1])

In [4]:
x_ptr = x.send(bob)
y_ptr = y.send(bob)

Pointers to tensors do NOT actually hold data themselves.  
Instead, they simply contain metadata about a tensor (with data) stored on another machine.

There are two main attributes specific to pointers:

    x_ptr.location : bob, the location, a reference to the location that the pointer is pointing to
    x_ptr.id_at_location : <random integer>, the id where the tensor is stored at location

They are printed in the format ```<id_at_location>@<location>```

There are also other more generic attributes:

    x_ptr.id : <random integer>, the id of oui pointer tensor, it was allocated randomly
    x_ptr.owner : me, the worker which owns the pointer tensor, here it's the local worker, named "me"


In [5]:
print(x_ptr)
print(x_ptr.owner)
print(x_ptr.id)
print(x_ptr.location)
print(x_ptr.id_at_location)

(Wrapper)>[PointerTensor | me:78518534495 -> bob:51913804772]
<VirtualWorker id:me #objects:0>
78518534495
<VirtualWorker id:bob #objects:2>
51913804772


In [6]:
print(bob._objects)

{51913804772: tensor([1, 2, 3, 4, 5]), 88325629745: tensor([1, 1, 1, 1, 1])}


just like we can call .send() on a tensor, we can call .get() on a pointer to a tensor to get it back!!!

In [7]:
print(x_ptr.get())
print(y_ptr.get())

tensor([1, 2, 3, 4, 5])
tensor([1, 1, 1, 1, 1])


# Lesson: Introducing Remote Arithmetic

So, sending and receiving tensors from Bob is great, but this is hardly Deep Learning!  
**We want to be able to perform tensor operations on remote tensors.**  
Fortunately, You can just use pointers like you would normal tensors!

In [15]:
a = torch.tensor([1,2,3,4,5]).send(bob) # PointerTensor
b = torch.tensor([1,1,1,1,1]).send(bob) # PointerTensor

In [16]:
c = a + b

In [17]:
c # PointerTensor

(Wrapper)>[PointerTensor | me:79289844300 -> bob:78251341965]

In [32]:
c = c.get()
c

tensor([2, 3, 4, 5, 6])

In [23]:
d = torch.add(a,b)
d

(Wrapper)>[PointerTensor | me:80652524728 -> bob:30006431160]

In [24]:
d = d.get()
d

tensor([2, 3, 4, 5, 6])

**Variables (including backpropagation!)**

In [25]:
x = torch.tensor([1.,2,3,4,5], requires_grad=True).send(bob)
y = torch.tensor([1.,1,1,1,1], requires_grad=True).send(bob)

In [26]:
z = (x + y).sum()

In [27]:
z.backward()

(Wrapper)>[PointerTensor | me:67295083337 -> bob:84706118614]

In [28]:
x = x.get()

In [29]:
x

tensor([1., 2., 3., 4., 5.], requires_grad=True)

In [30]:
x.grad

tensor([1., 1., 1., 1., 1.])

# Project: Learn a Simple Linear Model

In this project, I'd like for you to create a simple linear model which will solve for the following dataset below. You should use only Variables and .backward() to do so (no optimizers or nn.Modules). Furthermore, you must do so with both the data and the model being located on Bob's machine.

In [31]:
input = torch.tensor([[1.,1],[0,1,],[1,0],[0,0]], requires_grad=True).send(bob)
target = torch.tensor([[1.],[1],[0],[0]], requires_grad=True).send(bob)

In [32]:
weights = torch.tensor([[0.],[0.]], requires_grad=True).send(bob)

In [33]:
for i in range(10):

    pred = input.mm(weights)

    loss = ((pred - target)**2).sum()

    loss.backward()

    weights.data.sub_(weights.grad * 0.1) # substract
    weights.grad *= 0

    print(loss.get().data)

tensor(2.)
tensor(0.5600)
tensor(0.2432)
tensor(0.1372)
tensor(0.0849)
tensor(0.0538)
tensor(0.0344)
tensor(0.0220)
tensor(0.0141)
tensor(0.0090)


# Lesson: Garbage Collection and Common Errors


In [34]:
bob = bob.clear_objects()

In [35]:
bob._objects

{}

In [37]:
x = torch.tensor([1,2,3,4,5]).send(bob)

In [38]:
bob._objects

{25625681727: tensor([1, 2, 3, 4, 5])}

In [39]:
del x

In [40]:
bob._objects

{}

In [42]:
x = torch.tensor([1,2,3,4,5]).send(bob)

In [43]:
bob._objects

{69114594752: tensor([1, 2, 3, 4, 5])}

In [44]:
x = "asdf"

In [45]:
bob._objects

{}

In [46]:
x = torch.tensor([1,2,3,4,5]).send(bob)

In [47]:
x

(Wrapper)>[PointerTensor | me:19432551340 -> bob:64163024763]

In [48]:
bob._objects

{64163024763: tensor([1, 2, 3, 4, 5])}

In [49]:
x = "asdf"

In [50]:
bob._objects

{64163024763: tensor([1, 2, 3, 4, 5])}

In [51]:
del x

In [52]:
bob._objects

{64163024763: tensor([1, 2, 3, 4, 5])}

In [53]:
bob = bob.clear_objects()
bob._objects

{}

In [55]:
for i in range(1000):
    x = torch.tensor([1,2,3,4,5]).send(bob)

In [56]:
bob._objects

{59669740021: tensor([1, 2, 3, 4, 5])}

In [69]:
x = torch.tensor([1,2,3,4,5]).send(bob)
y = torch.tensor([1,1,1,1,1]) # note that .send() is not here

In [70]:
z = x + y

TensorsNotCollocatedException: You tried to call a method involving two tensors where one tensor is actually locatedon another machine (is a PointerTensor). Call .get() on the PointerTensor or .send(bob) on the other tensor.

Tensor A: [PointerTensor | me:35393948329 -> bob:92709280858]
Tensor B: tensor([1, 1, 1, 1, 1])

In [73]:
alice = syft.VirtualWorker(hook, id="alice")
x = torch.tensor([1,2,3,4,5]).send(bob)
y = torch.tensor([1,1,1,1,1]).send(alice) # note that the recevier is different

In [74]:
z = x + y

TensorsNotCollocatedException: You tried to call __add__ involving two tensors which are not on the same machine! One tensor is on <VirtualWorker id:bob #objects:8> while the other is on <VirtualWorker id:alice #objects:5>. Use a combination of .move(), .get(), and/or .send() to co-locate them to the same machine.

# Lesson: Toy Federated Learning

Let's start by training a toy model the centralized way. This is about a simple as models get. We first need:

- a toy dataset
- a model
- some basic training logic for training a model to fit the data.

In [69]:
from torch import nn, optim

In [67]:
# A Toy Dataset
data = th.tensor([[1.,1],[0,1],[1,0],[0,0]], requires_grad=True)
target = th.tensor([[1.],[1], [0], [0]], requires_grad=True)

In [70]:
# A Toy Model
model = nn.Linear(2,1)

In [71]:
opt = optim.SGD(params=model.parameters(), lr=0.1)

In [85]:
def train(iterations=20):
    for iter in range(iterations):
        opt.zero_grad()

        pred = model(data)

        loss = ((pred - target)**2).sum()

        loss.backward()

        opt.step()

        print(loss.data)
        
train()

tensor(0.0860)
tensor(0.0586)
tensor(0.0402)
tensor(0.0278)
tensor(0.0194)
tensor(0.0136)
tensor(0.0096)
tensor(0.0069)
tensor(0.0049)
tensor(0.0036)
tensor(0.0026)
tensor(0.0019)
tensor(0.0014)
tensor(0.0010)
tensor(0.0008)
tensor(0.0006)
tensor(0.0004)
tensor(0.0003)
tensor(0.0002)
tensor(0.0002)


In [89]:
data_bob = data[0:2].send(bob)
target_bob = target[0:2].send(bob)

In [90]:
data_alice = data[2:4].send(alice)
target_alice = target[2:4].send(alice)

In [91]:
datasets = [(data_bob, target_bob), (data_alice, target_alice)]

In [122]:
def train(iterations=20):

    model = nn.Linear(2,1)
    opt = optim.SGD(params=model.parameters(), lr=0.1)
    
    for iter in range(iterations):

        for _data, _target in datasets:

            # send model to the data
            model = model.send(_data.location)

            # do normal training
            opt.zero_grad()
            pred = model(_data)
            loss = ((pred - _target)**2).sum()
            loss.backward()
            opt.step()

            # get smarter model back
            model = model.get()

            print(loss.get())

In [123]:
train()

tensor(0.0824, requires_grad=True)
tensor(0.0006, requires_grad=True)
tensor(5.0509e-06, requires_grad=True)
tensor(1.8660e-07, requires_grad=True)
tensor(1.2442e-07, requires_grad=True)
tensor(1.0333e-07, requires_grad=True)
tensor(8.5991e-08, requires_grad=True)
tensor(7.1573e-08, requires_grad=True)
tensor(5.9562e-08, requires_grad=True)
tensor(4.9569e-08, requires_grad=True)
tensor(4.1269e-08, requires_grad=True)
tensor(3.4340e-08, requires_grad=True)
tensor(2.8568e-08, requires_grad=True)
tensor(2.3789e-08, requires_grad=True)
tensor(1.9802e-08, requires_grad=True)
tensor(1.6471e-08, requires_grad=True)
tensor(1.3715e-08, requires_grad=True)
tensor(1.1405e-08, requires_grad=True)
tensor(9.4892e-09, requires_grad=True)
tensor(7.9073e-09, requires_grad=True)
tensor(6.5776e-09, requires_grad=True)
tensor(5.4857e-09, requires_grad=True)
tensor(4.5577e-09, requires_grad=True)
tensor(3.8027e-09, requires_grad=True)
tensor(3.1617e-09, requires_grad=True)
tensor(2.6215e-09, requires_grad=

# Lesson: Advanced Remote Execution Tools

In the last section we trained a toy model using Federated Learning. We did this by calling .send() and .get() on our model, sending it to the location of training data, updating it, and then bringing it back. However, at the end of the example we realized that we needed to go a bit further to protect people privacy. Namely, we want to average the gradients BEFORE calling .get(). That way, we won't ever see anyone's exact gradient (thus better protecting their privacy!!!)

But, in order to do this, we need a few more pieces:

- use a pointer to send a Tensor directly to another worker

And in addition, while we're here, we're going to learn about a few more advanced tensor operations as well which will help us both with this example and a few in the future!

In [163]:
bob.clear_objects()
alice.clear_objects()

<VirtualWorker id:bob #tensors:0>

<VirtualWorker id:alice #tensors:0>

In [167]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [170]:
x = x.send(alice)

In [171]:
bob._objects

{21931995238: tensor([1, 2, 3, 4, 5])}

In [172]:
alice._objects

{55299383133: (Wrapper)>[PointerTensor | alice:55299383133 -> bob:21931995238]}

In [174]:
y = x + x

In [175]:
y

(Wrapper)>[PointerTensor | me:48924169518 -> alice:48924169518]

In [176]:
bob._objects

{21931995238: tensor([1, 2, 3, 4, 5]),
 48924169518: tensor([ 2,  4,  6,  8, 10])}

In [177]:
alice._objects

{48924169518: (Wrapper)>[PointerTensor | alice:48924169518 -> bob:48924169518],
 55299383133: (Wrapper)>[PointerTensor | alice:55299383133 -> bob:21931995238]}

In [178]:
jon = sy.VirtualWorker(hook, id="jon")

In [186]:
bob.clear_objects()
alice.clear_objects()

x = th.tensor([1,2,3,4,5]).send(bob).send(alice)
# y = th.tensor([1,2,3,4,5]).send(bob).send(jon)
# x + y ; produce error, ownership structure of x and y is different from each other

In [187]:
bob._objects

{10539507281: tensor([1, 2, 3, 4, 5])}

In [188]:
alice._objects

{40541026396: (Wrapper)>[PointerTensor | alice:40541026396 -> bob:10539507281]}

In [189]:
x = x.get()
x

(Wrapper)>[PointerTensor | me:40541026396 -> bob:10539507281]

In [190]:
bob._objects

{10539507281: tensor([1, 2, 3, 4, 5])}

In [191]:
alice._objects # after x.get(), alice's object gone

{}

In [192]:
x = x.get()
x

tensor([1, 2, 3, 4, 5])

In [193]:
bob._objects # after x.get() again, bob's object gone

{}

In [194]:
bob.clear_objects()
alice.clear_objects()

x = th.tensor([1,2,3,4,5]).send(bob).send(alice)

In [195]:
bob._objects

{28843833290: tensor([1, 2, 3, 4, 5])}

In [196]:
alice._objects

{88814770977: (Wrapper)>[PointerTensor | alice:88814770977 -> bob:28843833290]}

In [197]:
del x

In [198]:
bob._objects

{}

In [199]:
alice._objects

{}

# Lesson: Pointer Chain Operations

In [212]:
bob.clear_objects()
alice.clear_objects()

<VirtualWorker id:alice #tensors:0>

## .move()
1) me.send(bob)   
2) me.move(alice) : object in bob will move into alice, bob loses object

In [213]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [214]:
bob._objects

{81966670653: tensor([1, 2, 3, 4, 5])}

In [215]:
alice._objects

{}

In [216]:
x.move(alice)

(Wrapper)>[PointerTensor | me:547575813 -> alice:547575813]

In [217]:
bob._objects

{}

In [218]:
alice._objects

{547575813: tensor([1, 2, 3, 4, 5])}

## .remote_get()

In [202]:
x = th.tensor([1,2,3,4,5]).send(bob).send(alice)

In [203]:
bob._objects

{75487377866: tensor([1, 2, 3, 4, 5])}

In [204]:
alice._objects

{94092707138: (Wrapper)>[PointerTensor | alice:94092707138 -> bob:75487377866]}

In [205]:
x.remote_get()

(Wrapper)>[PointerTensor | me:87205391815 -> alice:94092707138]

In [206]:
bob._objects

{}

In [207]:
alice._objects

{94092707138: tensor([1, 2, 3, 4, 5])}

In [208]:
x.move(bob)

(Wrapper)>[PointerTensor | me:87205391815 -> bob:87205391815]

In [209]:
x

(Wrapper)>[PointerTensor | me:87205391815 -> bob:87205391815]

In [210]:
bob._objects

{87205391815: tensor([1, 2, 3, 4, 5])}

In [211]:
alice._objects

{94092707138: tensor([1, 2, 3, 4, 5])}